# Song Feature and Lyric Analysis

### by Benjamin Fenison and Stephanie Myott-Beebe
#### (SIADS 591/592: Milestone I Project)

Our project uses audio features and lyrics to find similarities within sets of songs. The inspiration behind the project is a tool to give the user insight into creating a song akin to the input song(s). For example, a musician looking to write a hit R&B song may be interested in common features and language found in current chart-topping R&B songs.

For purposes of this project, we are going to analyze the top five current hit songs for each of the following music genres: country, R&B/hip-hop, and rock/alternative, as determined by the Billboard Top 100 charts. For the week of May 15, 2021, the top 5 songs by genre are:

**Country**

 - 'Forever After All' by Luke Combs
 - 'The Good Ones' by Gabby Barrett
 - 'Made for You' by Jake Owen
 - 'Hell of a View' by Eric Church
 - 'Breaking Up Was Easy in the 90s' by Sam Hunt


**R&B/Hip-Hop**

 - 'Leave the Door Open' by Silk Sonic (Bruno Mars & Anderson .Paak)
 - 'Peaches' by Justin Bieber ft. Daniel Caesar & Giveon
 - 'Rapstar' by Polo G
 - 'Astronaut in the Ocean' by Masked Wolf
 - 'Up' by Cardi B 


**Rock/Alternative**

 - 'Without You' by The Kid LAROI & Miley Cyrus 
 - 'Your Power' by Billie Eilish 
 - 'My Ex's Best Friend' by Machine Gun Kelly X blackbear
 - 'Mood' by 24kGoldn ft. iann dior
 - 'Therefore I Am' by Billie Eilish 

Steps:

 1) The user inputs up to five songs to analyze. 

 2) The inputs are run through Spotify's API to identify songs that are most similar to the input songs.
     - Spotify has a recommendation algorithm that can take up to five songs, artists, and/or genres as seeds.
     - The algorithm returns up to 100 songs that it determines are most similar to the seeds.

      
 3) In order to obtain a large enough sample of songs, the top X songs returned from the previous query are each run through Spotify's recommendation algorithm.
 4) In addition to the track and artist, we will download additional features for each song from Spotify, including acousticness, danceability, energy, instrumentalness, and speechiness.
 5) Once we have songs and song features, each song will be run through the Genius API to obtain lyrics.
     - The Genius API does not have a mechanism for returning the lyrics of a song.
     - We are using the LyricsGenius package to interface with the Genius API. LyricsGenius uses the BeautifulSoup library to scrape song lyrics from a page's HTML.
 
 6) The data from Spotify and from Genius will be joined together in a pandas dataframe by track and artist.

Our ultimate goal is to create an application that would allow a user to input target songs




Our methodology is as follows:

 1) 

We are going to compare the current top 5 songs from the following genres:

 - Country
 - R&B/Hip-Hop
 - Rock/Alternative



#### Spotify API

Every time the recommendations function is run, the API returns up to 100 songs similar to the input songs, based on Spotify's bart algorithm. Based on research, the algorithm uses three functions:

 1. Natural language processing
 2. Raw audio analyzation
 3. Collaborative filtering

However, we do not know the exact manner in which analysis is performed, the threshold that must be met in order for the algorithm to return a song as being similar to the seed tracks, or the ranking of similarity of those songs that are returned. This is evident from the fact that the algorithm does not return the same songs each time the algorithm is ran. As a result, the analysis is not 

We are using Spotify's algorithm to pull similar songs but are performing some analysis of our own to increase our list of similar songs (in part, for purposes of lyric analysis).

***INSERT WHY WE CHOSE EUCLIDEAN DISTANCE***